Goal here is to record averages for each team for each season.

In [1]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [2]:
data = pd.read_sql("SELECT * FROM sports_data.game_stats_v2", conn)

In [3]:
data.shape[0]

105348

In [4]:
data.columns

Index(['date', 'game_type', 'team', 'conference', 'opponent', 'location',
       'result', 'adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference', 'season', 'Win_Loss', 'score', 'opponent_score'],
      dtype='object')

In [5]:
data['game_type'].unique()

array(['Non-Conference', 'Conference', 'Conference Tourney',
       'Post-Season Tourney'], dtype=object)

In [6]:
pre_tourney = data[data['game_type'].isin(['Non-Conference', 'Conference', 'Conference Tourney'])]

In [7]:
pre_tourney.columns[7:-4]

Index(['adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference'],
      dtype='object')

In [17]:
season_avgs = pre_tourney.groupby(['team', 'season'], as_index=False).mean(numeric_only=True)

In [20]:
season_avgs

,team,season,adj_offense,adj_defense,offensive_efficiency,effective_fg_percent,turnover_percent,offensive_rebound_percent,free_throw_rate,opponent_offensive_efficiency,opponent_effective_fg_percent,opponent_turnover_percent,opponent_offensive_rebound_percent,opponent_free_throw_rate
0,Abilene Christian,2015,91.464286,111.442857,93.303571,47.664286,19.896429,23.957143,28.996429,111.003571,55.500000,20.717857,33.925000,45.482143
1,Abilene Christian,2016,96.292593,111.837037,99.648148,50.659259,19.207407,24.259259,40.581481,109.218519,54.492593,19.229630,27.662963,47.922222
2,Abilene Christian,2017,96.972000,106.736000,98.128000,52.456000,21.020000,22.952000,37.024000,105.164000,52.820000,21.348000,31.464000,44.880000
3,Abilene Christian,2018,97.044444,106.859259,99.988889,51.103704,20.211111,26.144444,34.214815,103.007407,51.929630,21.700000,29.403704,44.374074
4,Abilene Christian,2019,102.234483,100.989655,107.093103,52.468966,17.106897,27.441379,34.882759,96.751724,49.755172,23.303448,27.431034,38.944828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,Youngstown St.,2020,100.764516,106.761290,103.490323,47.606452,17.854839,35.290323,30.693548,104.419355,50.048387,16.603226,29.222581,31.519355
3536,Youngstown St.,2021,101.943478,109.313043,106.104348,48.904348,16.613043,32.586957,32.230435,108.591304,54.630435,18.582609,28.830435,26.713043
3537,Youngstown St.,2022,102.563333,109.223333,106.810000,51.603333,17.080000,29.613333,30.163333,105.186667,51.266667,17.340000,27.963333,29.913333
3538,Youngstown St.,2023,110.119355,108.919355,115.670968,55.532258,15.661290,30.464516,30.196774,104.345161,52.264516,18.054839,28.058065,26.825806


In [19]:
season_avgs.to_sql('season_averages_by_team', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', index=False, schema='sports_data')

540